<a href="https://colab.research.google.com/github/bubai-jkc/cub_resnet50/blob/main/cub_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#dataset-----https://www.kaggle.com/datasets/wenewone/cub2002011

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d wenewone/cub2002011

Dataset URL: https://www.kaggle.com/datasets/wenewone/cub2002011
License(s): CC0-1.0
 99% 1.48G/1.49G [00:19<00:00, 155MB/s]
100% 1.49G/1.49G [00:19<00:00, 83.3MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cub2002011.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
pip install split-folders

In [5]:
import splitfolders
input_folder = "/content/CUB_200_2011/images"
output = "/content/cub"
splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8,.2))


Copying files: 11796 files [00:24, 481.30 files/s]


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

train_dir = '/content/cub/train'
test_dir = '/content/cub/val'

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 9416 images belonging to 200 classes.
Found 2380 images belonging to 200 classes.


In [7]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(200, activation='softmax')
])


94765736/94765736 [==============================] - 1s 0us/step


In [9]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               102600    
                                                                 
Total params: 24739400 (94.37 MB)
Trainable params: 1151688 (4.39 MB)
Non-trainable params: 23587712 (89.98 MB)
__________

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=9,
    validation_data=test_generator
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/9
295/295 [==============================] - 2188s 7s/step - loss: 5.3244 - accuracy: 0.0039 - val_loss: 5.2961 - val_accuracy: 0.0059
Epoch 2/9
295/295 [==============================] - 2184s 7s/step - loss: 5.2985 - accuracy: 0.0042 - val_loss: 5.2979 - val_accuracy: 0.0050
Epoch 3/9
295/295 [==============================] - 2198s 7s/step - loss: 5.2991 - accuracy: 0.0029 - val_loss: 5.3040 - val_accuracy: 0.0050
Epoch 4/9
261/295 [=========================>....] - ETA: 3:24 - loss: 5.2984 - accuracy: 0.0038

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()
